# Create Temporary Tables Using SQL Files

This tutorial demonstrates how to create temporary tables in athena using `pydbtools.read_sql_queries`. This is an amended version of [create_temporary_version.ipynb](create_temporary_version.ipynb).

## Setup

Just run this script to create the source database so we can use it for our example.

In [1]:
import os
import pandas as pd
import awswrangler as wr
import pydbtools as pydb

In [2]:
# setup your own testing area (set foldername = GH username)
foldername = "mratford" # GH username
foldername = foldername.lower().replace("-","_")

In [3]:
bucketname = "alpha-everyone"
s3_base_path = f"s3://{bucketname}/{foldername}/"

db_name = f"aws_example_{foldername}"
source_db_base_path = f"s3://{bucketname}/{foldername}/source_db/"

# Delete all the s3 files in a given path
if wr.s3.list_objects(s3_base_path):
    print("deleting objs")
    wr.s3.delete_objects(s3_base_path)

# Delete the database if it exists
df_dbs = wr.catalog.databases(None)
if db_name in df_dbs["Database"].to_list():
    print(f"{db_name} found deleting")
    wr.catalog.delete_database(
        name=db_name
    )

# Setup source database
# Create the database
wr.catalog.create_database(db_name)

# Iterate through the tables in data/ and write them to our db using awswrangler
for table_name in ["department", "employees", "sales"]:
    
    df = pd.read_csv(f"data/{table_name}.csv")
    table_path = os.path.join(source_db_base_path, f"{table_name}/")
    wr.s3.to_parquet(
        df=df,
        path=table_path,
        index=False,
        dataset=True, # True allows the other params below i.e. overwriting to db.table
        database=db_name,
        table=table_name,
        mode="overwrite",
    )

## Task

We are going to create a table that shows total sales per employee using all 3 tables.

In [4]:
pydb.read_sql_query(f"SELECT * FROM {db_name}.employees LIMIT 5", ctas_approach=False)

,employee_id,sex,forename,surname,department_id,manager_id
0,1,M,Dexter,Mitchell,1.0,17.0
1,2,F,Summer,Bennett,1.0,17.0
2,3,M,Pip,Carter,1.0,17.0
3,4,F,Bella,Long,1.0,17.0
4,5,F,Lexie,Perry,NaN,17.0


In [5]:
pydb.read_sql_query(f"SELECT * FROM {db_name}.department LIMIT 5", ctas_approach=False)

,department_id,department_name
0,1,Sales
1,2,Admin
2,3,Management
3,4,Technical
4,5,Maintenance


In [6]:
pydb.read_sql_query(f"SELECT * FROM {db_name}.sales LIMIT 5", ctas_approach=False)

,employee_id,qtr,sales
0,1,1,768.17
1,2,1,391.98
2,3,1,406.36
3,4,1,816.25
4,5,1,437.05


pydbtools has a `read_sql_queries function` that allows you to create temporary tables within SQL which you can refer to in a `__temp__` database.

**First create a total_sales table:**

In [7]:
sql = f"""
CREATE TEMP TABLE total_sales AS
SELECT employee_id, sum(sales) as total_sales
FROM {db_name}.sales
GROUP BY employee_id;
"""
print(sql)


CREATE TEMP TABLE total_sales AS
SELECT employee_id, sum(sales) as total_sales
FROM aws_example_mratford.sales
GROUP BY employee_id;



**Then create a table of employee names from the sales department:**

In [8]:
sql += f"""
CREATE TEMP TABLE sales_employees AS
SELECT e.employee_id, e.forename, e.surname, d.department_name
FROM {db_name}.employees AS e
LEFT JOIN {db_name}.department AS d
ON e.department_id = d.department_id
WHERE e.department_id = 1;
"""
print(sql)


CREATE TEMP TABLE total_sales AS
SELECT employee_id, sum(sales) as total_sales
FROM aws_example_mratford.sales
GROUP BY employee_id;

CREATE TEMP TABLE sales_employees AS
SELECT e.employee_id, e.forename, e.surname, d.department_name
FROM aws_example_mratford.employees AS e
LEFT JOIN aws_example_mratford.department AS d
ON e.department_id = d.department_id
WHERE e.department_id = 1;



**Finally return our final tables**

Note that more than one select statement can be used so the function returns an iterator yielding the results of each select.

In [9]:
sql += f"""
SELECT se.*, ts.total_sales
FROM __temp__.sales_employees AS se
INNER JOIN __temp__.total_sales AS ts
ON se.employee_id = ts.employee_id;

SELECT forename, surname, sum(s.sales) as q1_sales
FROM __temp__.sales_employees AS se
LEFT JOIN {db_name}.sales AS s
ON se.employee_id = s.employee_id
GROUP BY forename, surname;
"""
print(sql)


CREATE TEMP TABLE total_sales AS
SELECT employee_id, sum(sales) as total_sales
FROM aws_example_mratford.sales
GROUP BY employee_id;

CREATE TEMP TABLE sales_employees AS
SELECT e.employee_id, e.forename, e.surname, d.department_name
FROM aws_example_mratford.employees AS e
LEFT JOIN aws_example_mratford.department AS d
ON e.department_id = d.department_id
WHERE e.department_id = 1;

SELECT se.*, ts.total_sales
FROM __temp__.sales_employees AS se
INNER JOIN __temp__.total_sales AS ts
ON se.employee_id = ts.employee_id;

SELECT forename, surname, sum(s.sales) as q1_sales
FROM __temp__.sales_employees AS se
LEFT JOIN aws_example_mratford.sales AS s
ON se.employee_id = s.employee_id
GROUP BY forename, surname;



In [10]:
total_sales, q1_sales = tuple(pydb.read_sql_queries(sql))

In [11]:
total_sales

,employee_id,forename,surname,department_name,total_sales
0,1,Dexter,Mitchell,Sales,2911.65
1,2,Summer,Bennett,Sales,1785.73
2,3,Pip,Carter,Sales,2590.60
3,4,Bella,Long,Sales,2996.54
4,6,Robert,Roberts,Sales,2207.77
5,7,Iris,Alexander,Sales,2465.13
6,9,Evan,Carter,Sales,2279.84
7,10,Lauren,Powell,Sales,1935.67
8,11,Alice,James,Sales,3092.89
9,12,Owen,Scott,Sales,2286.28


In [12]:
q1_sales

,forename,surname,q1_sales
0,Lucas,Moore,2251.47
1,Sophie,Morris,1931.27
2,Kitty,Russell,2461.53
3,Sebastian,Hall,2547.63
4,Maisie,Torres,1688.76
5,Lola,Watson,1326.88
6,Eva,Peterson,2851.36
7,Pip,Carter,2590.60
8,Madison,Kelly,2302.02
9,Dexter,Mitchell,2911.65


In [13]:
### Clean up

# Delete all the s3 files in a given path
if wr.s3.list_objects(s3_base_path):
    print("deleting objs")
    wr.s3.delete_objects(s3_base_path)

# Delete the database if it exists
df_dbs = wr.catalog.databases(None)
if db_name in df_dbs["Database"].to_list():
    print(f"{db_name} found deleting")
    wr.catalog.delete_database(
        name=db_name
    )

deleting objs
aws_example_mratford found deleting
